Using CNN


In [5]:
import numpy as np
import pickle
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models,layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import cv2
import os



In [ ]:
from google.colab import files
files.upload()  # This will prompt you to upload the kaggle.json file


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shadowmonarchsunjin","key":"42bc0f9d1cc262c501d04c8a6d0550fc"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d manjilkarki/deepfake-and-real-images


Dataset URL: https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images
License(s): unknown
100% 1.68G/1.68G [00:08<00:00, 145MB/s]
100% 1.68G/1.68G [00:08<00:00, 223MB/s]


In [ ]:
!unzip deepfake-and-real-images.zip


Streaming output truncated to the last 5000 lines.
  inflating: Dataset/Validation/Real/real_5499.jpg  
  inflating: Dataset/Validation/Real/real_55.jpg  
  inflating: Dataset/Validation/Real/real_550.jpg  
  inflating: Dataset/Validation/Real/real_5500.jpg  
  inflating: Dataset/Validation/Real/real_5501.jpg  
  inflating: Dataset/Validation/Real/real_5502.jpg  
  inflating: Dataset/Validation/Real/real_5503.jpg  
  inflating: Dataset/Validation/Real/real_5504.jpg  
  inflating: Dataset/Validation/Real/real_5505.jpg  
  inflating: Dataset/Validation/Real/real_5506.jpg  
  inflating: Dataset/Validation/Real/real_5507.jpg  
  inflating: Dataset/Validation/Real/real_5508.jpg  
  inflating: Dataset/Validation/Real/real_5509.jpg  
  inflating: Dataset/Validation/Real/real_551.jpg  
  inflating: Dataset/Validation/Real/real_5510.jpg  
  inflating: Dataset/Validation/Real/real_5511.jpg  
  inflating: Dataset/Validation/Real/real_5512.jpg  
  inflating: Dataset/Validation/Real/real_5513.jpg  

In [6]:
def load_images_from_folder(folder, label):
    images = []
    labels = []
    #path = list(os.listdir(folder))
    #size = int(len(path)/2)
    #print(size)
    #half = path[:size]
    #print(len(half))
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (128, 128))  # Resize images to 128x128 pixels
            images.append(img)
            labels.append(label)
    return images, labels

real_images, real_labels = load_images_from_folder('Dataset/Train/Real', 0)
fake_images, fake_labels = load_images_from_folder('Dataset/Train/Fake', 1)

# Combine real and fake data
print(len(real_images))
print(len(fake_images))

images = np.array(real_images + fake_images)
labels = np.array(real_labels + fake_labels)

70001
70001


In [8]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_test = scaler.fit_transform(x_test)
x_train = x_train / 255.0  # Normalize pixel values
x_test = x_test / 255.0


In [9]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [10]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
3501/3501 [==============================] - 401s 114ms/step - loss: 0.3076 - accuracy: 0.8594 - val_loss: 0.1920 - val_accuracy: 0.9178
Epoch 2/10
3501/3501 [==============================] - 383s 109ms/step - loss: 0.1595 - accuracy: 0.9354 - val_loss: 0.1384 - val_accuracy: 0.9450
Epoch 3/10
3501/3501 [==============================] - 377s 108ms/step - loss: 0.1260 - accuracy: 0.9494 - val_loss: 0.1203 - val_accuracy: 0.9526
Epoch 4/10
3501/3501 [==============================] - 371s 106ms/step - loss: 0.1074 - accuracy: 0.9569 - val_loss: 0.1128 - val_accuracy: 0.9564
Epoch 5/10
3501/3501 [==============================] - 372s 106ms/step - loss: 0.0955 - accuracy: 0.9620 - val_loss: 0.1081 - val_accuracy: 0.9574
Epoch 6/10
3501/3501 [==============================] - 368s 105ms/step - loss: 0.0872 - accuracy: 0.9654 - val_loss: 0.1031 - val_accuracy: 0.9621
Epoch 7/10
3501/3501 [==============================] - 372s 106ms/step - loss: 0.0794 - accuracy: 0.9686 - val_

In [11]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

876/876 [==============================] - 25s 29ms/step - loss: 0.1005 - accuracy: 0.9635
Test Accuracy: 96.35%


In [12]:
import sys

print(sys.getsizeof(model))

48


In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image
img_path = '20210119_210738.jpg'
img = image.load_img(img_path, target_size=(128, 128))  # Resize to 224x224 if that's what your model expects
img_array = image.img_to_array(img)  # Convert to array
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize the image

# Load your pre-trained deep fake analysis model
#model = tf.keras.models.load_model('path_to_your_model.h5')

# Pass the image through the model
predictions = model.predict(img_array)

# Interpret the results
print(predictions)

1/1 [==============================] - 0s 92ms/step
[[0.29960442]]


In [15]:
!kaggle datasets download -d dagnelies/deepfake-faces

Dataset URL: https://www.kaggle.com/datasets/dagnelies/deepfake-faces
License(s): other
 98% 423M/433M [00:02<00:00, 146MB/s]
100% 433M/433M [00:02<00:00, 176MB/s]


In [16]:
!unzip deepfake-faces.zip

Streaming output truncated to the last 5000 lines.
  inflating: faces_224/yppwksjuui.jpg  
  inflating: faces_224/yppybofyww.jpg  
  inflating: faces_224/ypqjxdwawy.jpg  
  inflating: faces_224/ypqsxalhkw.jpg  
  inflating: faces_224/ypqxbbydxe.jpg  
  inflating: faces_224/ypqzrivawi.jpg  
  inflating: faces_224/yprdarrkit.jpg  
  inflating: faces_224/yprhthcacq.jpg  
  inflating: faces_224/yprmkhffdi.jpg  
  inflating: faces_224/yprnmqkhga.jpg  
  inflating: faces_224/yprqkgcoid.jpg  
  inflating: faces_224/yprsoygtjn.jpg  
  inflating: faces_224/yprxcbpmaj.jpg  
  inflating: faces_224/yprxtgidkk.jpg  
  inflating: faces_224/ypsbicsrsf.jpg  
  inflating: faces_224/ypsfvpanpn.jpg  
  inflating: faces_224/ypsgzbgbaz.jpg  
  inflating: faces_224/ypskfnxajx.jpg  
  inflating: faces_224/ypskxfumwo.jpg  
  inflating: faces_224/ypslvqfsrd.jpg  
  inflating: faces_224/ypsmruzzyt.jpg  
  inflating: faces_224/ypsqgodnrn.jpg  
  inflating: faces_224/ypsydrqqsl.jpg  
  inflating: faces_224/yptcha

In [17]:
test_real_images, test_real_labels = load_images_from_folder('Dataset/Test/Real', 0)
test_fake_images, test_fake_labels = load_images_from_folder('Dataset/Test/Fake', 1)

# Combine real and fake data
print(len(test_real_images))
print(len(test_fake_images))

test_images = np.array(test_real_images + test_fake_images)
test_labels = np.array(test_real_labels + test_fake_labels)

5413
5492


In [18]:
loss,accuracy = model.evaluate(test_images,test_labels)

341/341 [==============================] - 10s 28ms/step - loss: 53.7971 - accuracy: 0.8828


In [19]:
print(accuracy)

0.8828060626983643


In [22]:
print(model.evaluate(x_test,y_test))

876/876 [==============================] - 25s 29ms/step - loss: 0.1005 - accuracy: 0.9635
[0.10051394999027252, 0.9635012745857239]


In [23]:
import pickle

with open('model_v2.pickle','wb') as file:
  pickle.dump(model,file)

In [24]:
with open('model_v2.pickle','rb') as file:
  model_v2 = pickle.load(file)

In [25]:
loss,accuracy = model_v2.evaluate(x_test,y_test)

876/876 [==============================] - 24s 28ms/step - loss: 0.1005 - accuracy: 0.9635


In [26]:
print(accuracy)

0.9635012745857239


In [27]:
# Save the model
model.save('model_v2.h5')

# Load the model
from keras.models import load_model
model = load_model('model_v2.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
